In [1]:
import requests
import pandas as pd
import numpy as np
import timeit

### Steps to follow 
1. Get csv data of **IQA** 
2. Explore the data set
3. Filter the data by dept
4. analyse / visualize the data of each dept individually

In [3]:
# Read csv file that contains each code_zone of the 13 departments of Occitanie
dept = pd.read_excel("/home/zack/projet_pollution_occitanie/data/dept.xlsx")

# Get  13 clean (i.e doesn't contain empty values) lists that contains the codes of zones of each departments from our data frame "dept"
dept_lists = [dept[col].dropna().tolist() for col in dept.columns]

# From the "dept" df get the names of each department 
dept_names = dept.columns.to_list()

# Create a dictionary that assign each dept to it's codes of zone   
data = dict(zip(dept_names,dept_lists))

In [27]:
dept

,Averyron,Tarn-et garonne,Hautes-Pyrénées,Pyrénées-Orientales,Tarn,Hérlaut,Lot,Lozère,Gers,Gard,Ariège,Aude,Haute-Garonne
0,200067171,248200099.0,200069300.0,200027183.0,248100737.0,243400017.0,200023737.0,244800405.0,200066926.0,243000643.0,200067791.0,241100593.0,243100518.0
1,241200807,200066652.0,200072106.0,200043602.0,248100430.0,200066355.0,200066371.0,200069185.0,200023620.0,200066918.0,200066231.0,200035715.0,200068641.0
2,200067478,200066322.0,200070787.0,246600282.0,200066124.0,243400769.0,200067361.0,244800470.0,243200391.0,200034692.0,200067940.0,200035863.0,243100633.0
3,241200641,200066884.0,246500482.0,200070365.0,200040905.0,243400819.0,244600433.0,200069268.0,243200417.0,243000585.0,240900464.0,200071926.0,200072643.0
4,241200914,248200057.0,200070811.0,200049211.0,200034023.0,243400470.0,244600482.0,200069151.0,243200458.0,200034379.0,200066223.0,200035855.0,243100781.0
5,241200765,248200016.0,200070803.0,246600373.0,248100158.0,243400520.0,244600532.0,200069128.0,200034726.0,243000593.0,200044469.0,200035707.0,200071298.0
6,241200567,200067122.0,200070829.0,246600449.0,200034056.0,200022986.0,200039519.0,200006930.0,200072320.0,243000569.0,240900431.0,200043776.0,200068815.0
7,200067064,248200065.0,200070795.0,246600415.0,200066561.0,243400694.0,200035327.0,200069102.0,200042372.0,243000684.0,200066363.0,200042463.0,200071314.0
8,200068484,200040418.0,246500573.0,246600548.0,200034049.0,243400355.0,244600573.0,200069136.0,243200599.0,243000296.0,NaN,243100567.0,200068807.0
9,241200658,248200107.0,246401756.0,246600399.0,200066553.0,243400488.0,200069383.0,200069144.0,243200409.0,200034411.0,NaN,200070365.0,200066819.0


### Get the csv file

In [40]:
# Read the csv file 'indice de qualité de l'air'
df = pd.read_csv("/home/zack/projet_pollution_occitanie/data/IQA.csv", usecols=['code_no2',
 'code_o3',
 'code_pm10',
 'code_pm25',
 'code_qual',
 'code_so2',
 'code_zone',
 'coul_qual',
 'date_dif',
 'date_ech',
 'lib_qual',
 'lib_zone'])

We want to know how many days we have in this dataset

In [41]:
days_num = df['date_ech'].value_counts().size
start_date = df['date_ech'][0]
last_day = df['date_ech'][df.shape[0]-1]
print(f"Le nombre des jours inclus dans cetter analyse est {days_num} avec date de début {start_date} et jusqu'à {last_day}")

Le nombre des jours inclus dans cetter analyse est 688 avec date de début 2023/11/19 00:00:00+00 et jusqu'à 2022/01/01 00:00:00+00


In [42]:
# Create a copy to 
df2 = df.copy()


In [43]:
df2

,date_ech,code_qual,lib_qual,coul_qual,date_dif,code_zone,lib_zone,code_no2,code_so2,code_o3,code_pm10,code_pm25
0,2023/11/19 00:00:00+00,2,Moyen,#50CCAA,2023/11/18 09:00:00+00,200066223,CC Arize Lèze,1,1,2,1,2
1,2023/11/19 00:00:00+00,2,Moyen,#50CCAA,2023/11/18 09:00:00+00,200040905,CC Carmausin-Ségala,1,1,2,2,2
2,2023/11/19 00:00:00+00,2,Moyen,#50CCAA,2023/11/18 09:00:00+00,241200542,CC du Réquistanais,1,1,2,1,1
3,2023/11/19 00:00:00+00,2,Moyen,#50CCAA,2023/11/18 09:00:00+00,246600464,CC Capcir Haut-Conflent,2,1,2,1,2
4,2023/11/19 00:00:00+00,2,Moyen,#50CCAA,2023/11/18 09:00:00+00,200042372,CC des Coteaux Arrats Gimone,1,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
112827,2022/01/01 00:00:00+00,2,Moyen,#50CCAA,2022/01/02 09:00:00+00,200034957,CC du Frontonnais,2,1,1,2,2
112828,2022/01/01 00:00:00+00,2,Moyen,#50CCAA,2022/01/02 09:00:00+00,200034379,CC Pays d'Uzès,1,1,1,1,1
112829,2022/01/01 00:00:00+00,2,Moyen,#50CCAA,2022/01/02 09:00:00+00,200069128,CC Mont Lozère,1,1,1,1,2
112830,2022/01/01 00:00:00+00,2,Moyen,#50CCAA,2022/01/02 09:00:00+00,240900431,CC du Pays de Tarascon,1,1,2,1,1


In [44]:
dept

,Averyron,Tarn-et garonne,Hautes-Pyrénées,Pyrénées-Orientales,Tarn,Hérlaut,Lot,Lozère,Gers,Gard,Ariège,Aude,Haute-Garonne
0,200067171,248200099.0,200069300.0,200027183.0,248100737.0,243400017.0,200023737.0,244800405.0,200066926.0,243000643.0,200067791.0,241100593.0,243100518.0
1,241200807,200066652.0,200072106.0,200043602.0,248100430.0,200066355.0,200066371.0,200069185.0,200023620.0,200066918.0,200066231.0,200035715.0,200068641.0
2,200067478,200066322.0,200070787.0,246600282.0,200066124.0,243400769.0,200067361.0,244800470.0,243200391.0,200034692.0,200067940.0,200035863.0,243100633.0
3,241200641,200066884.0,246500482.0,200070365.0,200040905.0,243400819.0,244600433.0,200069268.0,243200417.0,243000585.0,240900464.0,200071926.0,200072643.0
4,241200914,248200057.0,200070811.0,200049211.0,200034023.0,243400470.0,244600482.0,200069151.0,243200458.0,200034379.0,200066223.0,200035855.0,243100781.0
5,241200765,248200016.0,200070803.0,246600373.0,248100158.0,243400520.0,244600532.0,200069128.0,200034726.0,243000593.0,200044469.0,200035707.0,200071298.0
6,241200567,200067122.0,200070829.0,246600449.0,200034056.0,200022986.0,200039519.0,200006930.0,200072320.0,243000569.0,240900431.0,200043776.0,200068815.0
7,200067064,248200065.0,200070795.0,246600415.0,200066561.0,243400694.0,200035327.0,200069102.0,200042372.0,243000684.0,200066363.0,200042463.0,200071314.0
8,200068484,200040418.0,246500573.0,246600548.0,200034049.0,243400355.0,244600573.0,200069136.0,243200599.0,243000296.0,NaN,243100567.0,200068807.0
9,241200658,248200107.0,246401756.0,246600399.0,200066553.0,243400488.0,200069383.0,200069144.0,243200409.0,200034411.0,NaN,200070365.0,200066819.0


Filter the data frame by department 

In [45]:
# Create a data frame for each department :
# Ariège
df_Ariège = df2[df2['code_zone'].isin(data['Ariège'])]
# Aude
df_Aude = df2[df2['code_zone'].isin(data['Aude'])]
# Averyron
df_Averyron = df2[df2['code_zone'].isin(data['Averyron'])]
# Gard
df_Gard = df2[df2['code_zone'].isin(data['Gard'])]
# Gers 
df_Gers = df2[df2['code_zone'].isin(data['Gers'])]
# Hautes_Pyrénées
df_Hautes_Pyrénées = df2[df2['code_zone'].isin(data['Hautes-Pyrénées'])]
# Hérlaut
df_Hérlaut = df2[df2['code_zone'].isin(data['Hérlaut'])]
#Lot
df_Lot = df2[df2['code_zone'].isin(data['Lot'])]
# Lozère
df_Lozère = df2[df2['code_zone'].isin(data['Lozère'])]
# Pyrénées_Orientales
df_Pyrénées_Orientales = df2[df2['code_zone'].isin(data['Pyrénées-Orientales'])]
# Tarn
df_Tarn = df2[df2['code_zone'].isin(data['Tarn'])]
# Tarn_garonne
df_Tarn_garonne = df2[df2['code_zone'].isin(data['Tarn-et garonne'])]
# Haute_Garonne
df_Haute_Garonne = df2[df2['code_zone'].isin(data['Haute-Garonne'])]

In [46]:
df_Hérlaut

,date_ech,code_qual,lib_qual,coul_qual,date_dif,code_zone,lib_zone,code_no2,code_so2,code_o3,code_pm10,code_pm25
16,2023/11/19 00:00:00+00,2,Moyen,#50CCAA,2023/11/18 09:00:00+00,243400355,CC du Clermontais,2,1,2,2,2
29,2023/11/19 00:00:00+00,2,Moyen,#50CCAA,2023/11/18 09:00:00+00,200071058,CC Les Avant-Monts,2,1,2,2,2
52,2023/11/19 00:00:00+00,2,Moyen,#50CCAA,2023/11/18 09:00:00+00,200042653,CC Sud-Hérault,2,1,2,2,2
62,2023/11/19 00:00:00+00,2,Moyen,#50CCAA,2023/11/18 09:00:00+00,243400520,CC du Pays de Lunel,2,1,2,2,2
70,2023/11/19 00:00:00+00,2,Moyen,#50CCAA,2023/11/18 09:00:00+00,200066355,CA Bassin de Thau,2,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
112771,2022/01/01 00:00:00+00,2,Moyen,#50CCAA,2022/01/02 09:00:00+00,200042646,CC Grand Orb en Languedoc,1,1,1,1,2
112774,2022/01/01 00:00:00+00,2,Moyen,#50CCAA,2022/01/02 09:00:00+00,243400769,CA de Béziers-Méditerranée,2,1,1,1,2
112795,2022/01/01 00:00:00+00,2,Moyen,#50CCAA,2022/01/02 09:00:00+00,243400819,CA Hérault-Méditerranée,1,1,1,1,2
112797,2022/01/01 00:00:00+00,2,Moyen,#50CCAA,2022/01/02 09:00:00+00,200066348,CC Minervois Saint-Ponais Orb-Jaur,1,1,1,1,2


In [47]:
df_Aude

,date_ech,code_qual,lib_qual,coul_qual,date_dif,code_zone,lib_zone,code_no2,code_so2,code_o3,code_pm10,code_pm25
28,2023/11/19 00:00:00+00,2,Moyen,#50CCAA,2023/11/18 09:00:00+00,200035715,CA Carcassonne,1,1,2,1,2
30,2023/11/19 00:00:00+00,2,Moyen,#50CCAA,2023/11/18 09:00:00+00,241100593,CA Le Grand Narbonne,1,1,2,1,2
34,2023/11/19 00:00:00+00,2,Moyen,#50CCAA,2023/11/18 09:00:00+00,200042463,CC de la Montagne Noire,1,1,2,1,2
47,2023/11/19 00:00:00+00,2,Moyen,#50CCAA,2023/11/18 09:00:00+00,200071926,CC du Limouxin,1,1,2,1,2
74,2023/11/19 00:00:00+00,2,Moyen,#50CCAA,2023/11/18 09:00:00+00,243100567,CC Lauragais Revel Sorezois,2,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
112750,2022/01/01 00:00:00+00,2,Moyen,#50CCAA,2022/01/02 09:00:00+00,200035863,"CC Région Lézignanaise, Corbières et Minervois",1,1,2,1,1
112766,2022/01/01 00:00:00+00,2,Moyen,#50CCAA,2022/01/02 09:00:00+00,200071926,CC du Limouxin,1,1,2,1,1
112783,2022/01/01 00:00:00+00,2,Moyen,#50CCAA,2022/01/02 09:00:00+00,200035707,CC Piège Lauragais Malepère,1,1,2,1,1
112809,2022/01/01 00:00:00+00,2,Moyen,#50CCAA,2022/01/02 09:00:00+00,200070365,CC Corbières Salanque Méditerranée,1,1,1,1,2
